# CUPED

У нас есть данные по транзакциям пользователей во время *теста и в предпериод*. Мы хотим просчитать результаты a/b-теста относительно продаж (gmv).  
Сравним p-value в обычном t-тесте без применения CUPED и p-value c применением *CUPED*

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import scipy.stats as sps

## Чтение и проверка данных

In [15]:
df = pd.read_csv('./data_ignored/hist_exp_data.csv')
df.head()

,gmv_hist,gmv_exp,group_name
0,200.78,123.19,test
1,363.80,134.49,control
2,39.93,116.72,control
3,150.99,177.67,control
4,208.93,65.30,test


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250000 entries, 0 to 249999
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   gmv_hist    250000 non-null  float64
 1   gmv_exp     250000 non-null  float64
 2   group_name  250000 non-null  object 
dtypes: float64(2), object(1)
memory usage: 5.7+ MB


Есть данные по клиентам и на тесте, и на предпериоде *без пропусков*

In [4]:
df.describe()

,gmv_hist,gmv_exp
count,250000.000000,250000.000000
mean,179.409521,149.096584
std,133.808405,111.837843
min,0.000000,0.000000
25%,72.707500,59.920000
50%,152.865000,126.430000
75%,259.210000,215.070000
max,946.870000,929.640000


Но есть нулевые значения. Посмотрим их

In [5]:
df[df.gmv_hist == 0].count()

gmv_hist      2
gmv_exp       2
group_name    2
dtype: int64

In [6]:
df[df.gmv_exp == 0].count()

gmv_hist      1
gmv_exp       1
group_name    1
dtype: int64

Мало клиентов, которые не совершали бы покупок в одном из периодов. Проигнорируем это

In [7]:
df.group_name.unique()

array(['test', 'control'], dtype=object)

## Расчитаем cuped-метрику

In [8]:
Y = df.gmv_exp
X_pre = df.gmv_hist
theta = np.cov(Y, X_pre)[0, 1] / np.var(X_pre)
Y_cuped = Y - theta * X_pre

df['gmv_exp_cuped'] = Y_cuped

In [9]:
df.head()

,gmv_hist,gmv_exp,group_name,gmv_exp_cuped
0,200.78,123.19,test,11.176763
1,363.80,134.49,control,-68.470531
2,39.93,116.72,control,94.443436
3,150.99,177.67,control,93.434127
4,208.93,65.30,test,-51.260043


## t-тест

*Разделим* участников на тест и контроль:

In [10]:
df_test = df[df.group_name == 'test']
df_control = df[df.group_name == 'control']

In [11]:
df.shape[0] == df_test.shape[0] + df_control.shape[0]

True

*Посчитаем t-тест* на исходных данных и на cuped-метрике:

In [12]:
test = df_test.gmv_exp
control = df_control.gmv_exp
test_cuped = df_test.gmv_exp_cuped
control_cuped = df_control.gmv_exp_cuped

In [14]:
_, pvalue_tt = sps.ttest_ind(test, control)  # t-тест с исходной метрикой
_, pvalue_cuped = sps.ttest_ind(test_cuped, control_cuped, equal_var=False)  # t-тест с cuped метрикой
print(f'p-value по результатам t-тестов: \nна исходной метрике - {pvalue_tt}, \nна cuped-метрике    - {pvalue_cuped}')

p-value по результатам t-тестов: 
на исходной метрике - 0.23267087319649138, 
на cuped-метрике    - 0.04671025976667769


Использование CUPED, *уменьшая дисперсию* метрики, позволяет увидеть меньшие эффекты.